# 1. Загрузка и подготовка данных

In [1]:
#importing libraries

import pandas as pd

from sklearn.model_selection import train_test_split

In [2]:
df1 = pd.read_csv('/datasets/geo_data_0.csv')
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


In [3]:
df2 = pd.read_csv('/datasets/geo_data_1.csv')
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


In [4]:
df3 = pd.read_csv('/datasets/geo_data_2.csv')
print(df3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


Загрузили данные по 3ем регионам. Во всех 3ех регионах данные без пропусков , видно сразу ненужный столбец для обучения модели - 'id'. Все данные в нужном для нас типе

# 2. Обучение и проверка модели

In [5]:
def split(df):
    train_df, valid_df = train_test_split(df, test_size = 0.25)
    target_train = train_df['product']
    features_train = train_df.drop(['id','product'], axis = 1)
    target_valid = valid_df['product']
    features_valid = valid_df.drop(['id','product'], axis = 1)
    return target_train, features_train, target_valid, features_valid

In [6]:
target_train1, features_train1, target_valid1, features_valid1 = split(df1)
target_train2, features_train2, target_valid2, features_valid2 = split(df2)
target_train3, features_train3, target_valid3, features_valid3 = split(df3)

In [7]:
from sklearn.linear_model import LinearRegression
#metrics
from sklearn.metrics import mean_squared_error

In [8]:
def prediction(target_train, features_train,target_valid,features_valid):
    model = LinearRegression()
    model.fit(features_train,target_train)
    prediction = pd.Series(model.predict(features_valid))
    rmse = mean_squared_error(target_valid,prediction)**0.5
    return prediction, rmse

In [9]:
prediction1, rmse1 = prediction(target_train1, features_train1, target_valid1, features_valid1)
prediction2, rmse2 = prediction(target_train2, features_train2, target_valid2, features_valid2)
prediction3, rmse3 = prediction(target_train3, features_train3, target_valid3, features_valid3)

In [10]:
print('Средний запас сырья в 1ом регионе:', prediction1.mean(),'RMSE:',rmse1)
print('Средний запас сырья в 2ом регионе:', prediction2.mean(),'RMSE:',rmse2)
print('Средний запас сырья в 3ем регионе:', prediction3.mean(),'RMSE:',rmse3)

Средний запас сырья в 1ом регионе: 92.5456564650271 RMSE: 37.915891976305886
Средний запас сырья в 2ом регионе: 69.06331020107707 RMSE: 0.8922686411368428
Средний запас сырья в 3ем регионе: 94.93395567371175 RMSE: 40.00960614117899


После обучения модели под каждый регион и предсказав целевую метрику получили ,что модель работает надежней всего во втором регионе , с наименьшей RMSE. Среднее значение запасов сырья однако там оказались наименьшие , в отличие от 1 и 3 регионов. Исходя из работы данной модели , можно однозначно составить представление о 2ом регионе ,хотя запасы там наименьшие.

# 3. Подготовка к расчёту прибыли

In [11]:
profit = 450000
budget = 10000000000
minimum = budget / 200 / profit

In [12]:
print('1 регион',df1['product'].mean())
print('2 регион', df2['product'].mean())
print('3 регион', df3['product'].mean())

1 регион 92.50000000000001
2 регион 68.82500000000002
3 регион 95.00000000000004


In [13]:
minimum

111.11111111111111

После подсчета минимума для безубыточной работы скважины получили ,что данное значение очень даже превосходит средние запасы сырья в каждом регионе и повышает шанс уйти в убыточную работу скважины

# 4. Расчёт прибыли и рисков 

In [14]:
import numpy as np
from scipy import stats as st

In [15]:
state = np.random.RandomState(12345)

In [16]:
#function to calculate revenue
def revenue(target, prediction, count):
    pred_sorted = prediction.sort_values(ascending=False)
    selected = target[pred_sorted.index][:count]
    return profit * selected.sum()

In [17]:
def revenue_risk(target, prediction):
    state = np.random.RandomState(12345)
    target = target.reset_index(drop=True)
    prediction = prediction.reset_index(drop=True)
    profits = []
    damages = []
    for i in range(1000):
        target_subsample = target.sample(n = 500, replace = True, random_state = state)
        pred_subsample = prediction[target_subsample.index]
        rev = revenue(target_subsample, pred_subsample, 200)
        profits.append(rev - budget)
        damages.append((rev-budget)<0)
    
    profits = pd.Series(profits)
    damages = pd.Series(damages)
    
    print('Средняя прибыль',profits.mean())
    confidence_interval = (profits.quantile(0.025), profits.quantile(0.975)) 
    print('Доверительный интевал', confidence_interval)
    print('Риски',damages.mean()*100,'%')
    print('Убытки', damages.mean())

In [18]:
revenue_risk(target_valid1, prediction1)

Средняя прибыль 444530632.5883531
Доверительный интевал (-87621890.60332264, 956352634.5722739)
Риски 5.4 %
Убытки 0.054


In [19]:
revenue_risk(target_valid2, prediction2)

Средняя прибыль 539931097.3635128
Доверительный интевал (123795678.04169388, 973268129.3422137)
Риски 0.5 %
Убытки 0.005


In [20]:
revenue_risk(target_valid3, prediction3)

Средняя прибыль 420942343.2359965
Доверительный интевал (-127501892.95933342, 958674764.1044736)
Риски 6.2 %
Убытки 0.062


Для каждого региона посчитали прибыль и убытки , доверительный интервал в 95% , а также и риски. В итоге по всем показателям можно определить лучшим регионом - второй регион. Средняя прибыль у него напорядок высше остальных 2ух , также убытки и риски минимальные относительно двух других ,что делает данный регион самым выгодным и самым благоприятными для вложений в скважины именно там. 